In [1]:
import pandas as pd
import os
import random

In [2]:
def add_col(df):
    pre = df["testId"][0]
    count = df["answerCode"][0]
    c = 1
    new = []

    for idx, answer in zip(df["testId"],df["answerCode"]):
        if pre != idx :
            pre = idx
            new.append(0)
            c = 1
            count = answer
        else :
            new.append(count/c)
            c += 1
            count += answer
    df['cum_correct'] = new
    return df

In [3]:
def add_last_problem(df):
    new = []
    pre = df['testId'][0]
    for idx in df['testId']:
        if pre != idx :
            new[-1]=-1
            pre = idx
        new.append(0)
    df['last_problem'] = new
    return df

In [4]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    df = add_col(df)
    df = add_last_problem(df)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']
    
    diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

    # 푸는 시간
    df['elapsed'] = diff
    df['elapsed'] = df['elapsed'].apply(lambda x : x if x <650 else None)
    # 앞 세자리 
    # 중간 세자리
    # 문제 번호
    df['grade']=df['testId'].apply(lambda x : int(x[1:4])//10)
    df['mid'] = df['testId'].apply(lambda x : int(x[-3:]))
    df['problem_number'] = df['assessmentItemID'].apply(lambda x : int(x[-3:]))
    
    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    df = pd.merge(df, student_grade, on=['userID','grade'], how="left")
    df = pd.merge(df, test_info, on=['testId'], how="left")
    
    df['problem_position'] = df['problem_number'] / df["problem_count"]
    
    return df

In [5]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
DATA_PATH = '/opt/ml/input/data/train_dataset/train_data.csv'
df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [6]:
student_grade = pd.read_csv("/opt/ml/input/data/DB/student_grade.csv")
test_info = pd.read_csv("/opt/ml/input/data/DB/test_info.csv")
correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
correct_t.columns = ["test_mean", 'test_sum']
correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
correct_k.columns = ["tag_mean", 'tag_sum']

student_grade = student_grade[["userID","grade","acc","count", "elapsed"]]
test_info = test_info[["testId","problem_number","type_number","acc","elapsed"]]

student_grade.columns = ["userID","grade","grade_acc","grade_count", "grade_elapsed"]
test_info.columns = ["testId","problem_count","type_count","test_acc","test_elapsed"]

In [7]:
df = feature_engineering(df)
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,cum_correct,last_problem,user_correct_answer,user_total_answer,...,tag_mean,tag_sum,grade_acc,grade_count,grade_elapsed,problem_count,type_count,test_acc,test_elapsed,problem_position
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1.0,0,NaN,0,...,0.955022,637.0,0.791908,346,36.533784,7,2,0.952667,20.842105,0.142857
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.0,0,1.0,1,...,0.913187,3040.0,0.791908,346,36.533784,7,2,0.952667,20.842105,0.285714
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1.0,0,2.0,2,...,0.913187,3040.0,0.791908,346,36.533784,7,2,0.952667,20.842105,0.428571
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1.0,0,3.0,3,...,0.913187,3040.0,0.791908,346,36.533784,7,2,0.952667,20.842105,0.571429
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1.0,0,4.0,4,...,0.913187,3040.0,0.791908,346,36.533784,7,2,0.952667,20.842105,0.714286


In [8]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
test_csv_file_path = '/opt/ml/input/data/train_dataset/test_data.csv'
test_df = pd.read_csv(test_csv_file_path, dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [9]:
# FEATURE ENGINEERING
test_df = feature_engineering(test_df)
test_df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,cum_correct,last_problem,user_correct_answer,user_total_answer,...,tag_mean,tag_sum,grade_acc,grade_count,grade_elapsed,problem_count,type_count,test_acc,test_elapsed,problem_position
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,1.00,0,NaN,0,...,0.641379,1023.0,0.655052,861,63.258152,7,4,0.549714,68.609658,0.142857
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,1.00,0,1.0,1,...,0.641379,1023.0,0.655052,861,63.258152,7,4,0.549714,68.609658,0.285714
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,1.00,0,2.0,2,...,0.670013,1535.0,0.655052,861,63.258152,7,4,0.549714,68.609658,0.428571
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.75,0,2.0,3,...,0.670013,1535.0,0.655052,861,63.258152,7,4,0.549714,68.609658,0.571429
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.60,0,2.0,4,...,0.568970,2314.0,0.655052,861,63.258152,7,4,0.549714,68.609658,0.857143


In [10]:
test_df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'cum_correct', 'last_problem', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'elapsed', 'grade', 'mid',
       'problem_number', 'test_mean', 'test_sum', 'tag_mean', 'tag_sum',
       'grade_acc', 'grade_count', 'grade_elapsed', 'problem_count',
       'type_count', 'test_acc', 'test_elapsed', 'problem_position'],
      dtype='object')

In [ ]:
blank = test_df[test_df['answerCode']==-1].copy()

In [11]:
from collections import defaultdict

In [12]:
def a(xx):
    d = defaultdict(set)
    for x in xx:
        grade = int(x[1:4])//10
        mid = int(x[-3:]) // 10
        d[grade].add(mid)
    return d

In [ ]:
# 비슷한 사람 : 정답률, elapsed가 비슷한 
# 비슷한 학습 경험 : seq length, testId가 비슷한
# 둘 다 

In [27]:
f = lambda x : list(x)[-1]
TRAINID = df.groupby(['userID']).agg({'testId':a,'answerCode':'count','elapsed':'mean','grade_acc':f, 'grade_count':f, 'grade':f, 'mid':f}).reset_index()

TRAINID.columns = ['userID', 'testId', 'count', 'elapsed', 'grade_acc','last_count', 'last_grade', 'last_mid']
TRAINID = TRAINID.sort_values(by=['count', 'last_grade','last_mid']).reset_index(drop=True)
TRAINID[:10]

,userID,testId,count,elapsed,grade_acc,last_count,last_grade,last_mid
0,7441,"{3: {7}, 4: {16}}",9,43.750000,1.000000,4,4,165
1,7414,"{4: {19}, 3: {2}, 2: {12}}",13,49.000000,0.600000,5,2,124
2,7252,"{3: {1}, 5: {19}, 4: {13}}",13,127.000000,0.500000,4,4,132
3,7390,"{2: {7}, 5: {19}}",13,22.454545,1.000000,4,5,193
4,7386,"{1: {1, 15}, 4: {13}}",14,28.000000,0.600000,10,1,155
5,6981,"{2: {19, 14}, 4: {19}}",14,26.666667,0.888889,9,2,143
6,7189,"{3: {14, 7}, 2: {17}}",14,36.833333,0.250000,4,2,179
7,7330,"{4: {9}, 1: {14}, 2: {17}}",14,22.333333,1.000000,4,2,179
8,7349,"{6: {18}, 4: {16}, 3: {12}}",14,68.500000,0.400000,5,3,126
9,7422,"{6: {9}, 3: {17, 4}}",14,20.500000,0.300000,10,3,171


In [28]:
f = lambda x : list(x)[-1]
TESTID = test_df.groupby(['userID']).agg({'testId':a,'answerCode':'count','elapsed':'mean','grade_acc':f,'grade_count':f, 'grade':f, 'mid':f}).reset_index()

TESTID.columns = ['userID', 'testId', 'count', 'elapsed', 'grade_acc', 'last_count', 'last_grade', 'last_mid']
TESTID = TESTID.sort_values(by=['count', 'last_grade','last_mid']).reset_index(drop=True)
TESTID[:10]

,userID,testId,count,elapsed,grade_acc,last_count,last_grade,last_mid
0,5965,"{3: {0, 18}, 2: {15}}",15,29.000000,0.500000,4,2,159
1,7404,"{2: {12}, 4: {10}, 3: {11}}",15,33.846154,0.500000,4,3,111
2,7033,"{4: {8, 18, 13}}",15,9.000000,0.214286,14,4,133
3,7354,"{1: {10}, 3: {14}, 4: {17}}",15,66.923077,0.250000,4,4,170
4,7373,"{3: {15}, 4: {18}, 5: {17}}",15,24.615385,1.000000,4,5,178
5,7416,"{8: {0}, 3: {16}, 5: {19}}",15,29.461538,0.666667,3,5,193
6,7417,"{1: {9, 10}, 5: {19}}",15,50.916667,0.666667,3,5,193
7,7216,"{2: {17, 18}, 3: {19}}",16,51.714286,0.400000,10,2,170
8,7439,"{4: {0, 19, 13}}",16,37.928571,0.733333,15,4,130
9,7146,"{1: {2}, 4: {19}, 6: {6}}",16,38.571429,0.000000,3,6,67


In [68]:
tdict = {}

# 적용 기준 


for _, row in TESTID.iterrows():
    # 1) 마지막 문제 유형이 같은가?
    candidate = TRAINID[TRAINID['last_grade']==row['last_grade']].copy()

    # 2) 이미 선택되지 않은 것 중
    candidate['c'] = candidate['userID'].apply(lambda x : x in tdict.values())
    candidate = candidate[candidate['c'] == False]
    
    # 3) 마지막 문제에 대한 학습이력 길이가 비슷한 상위 30개 중
    candidate['last_count'] = candidate['last_count'].apply(lambda x : abs(x-row['last_count']))
    candidate = candidate.sort_values(by='last_count').reset_index(drop=True)
    candidate = candidate[:min(candidate.shape[0],30)].copy()
    
    # 4) 시험지가 같거나 다르다면 해당 grade에 대한 정확도가 비슷한 것
    candidate['r_acc'] = (candidate['grade_acc'] - row['grade_acc']).apply(lambda x : x if x >=0 else -x)
    candidate['r_mid'] = (candidate['last_mid'] == row['last_mid']).astype(int)
    candidate = candidate.sort_values(by=['r_mid','r_acc']).reset_index(drop=True)
    tdict[row['userID']] = list(candidate['userID'])[0]

print('done!')
print(f'please check {len(tdict)} == {len(set(tdict.values()))}')

done!
744 == 744


In [69]:
val_index = sorted(list(tdict.values()))

In [70]:
validset = TRAINID[TRAINID["userID"].isin(val_index)]

In [71]:
validset

,userID,testId,count,elapsed,grade_acc,last_count,last_grade,last_mid
1,7414,"{4: {19}, 3: {2}, 2: {12}}",13,49.000000,0.600000,5,2,124
2,7252,"{3: {1}, 5: {19}, 4: {13}}",13,127.000000,0.500000,4,4,132
3,7390,"{2: {7}, 5: {19}}",13,22.454545,1.000000,4,5,193
5,6981,"{2: {19, 14}, 4: {19}}",14,26.666667,0.888889,9,2,143
8,7349,"{6: {18}, 4: {16}, 3: {12}}",14,68.500000,0.400000,5,3,126
...,...,...,...,...,...,...,...,...
6633,215,"{7: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",1326,53.961572,0.677140,923,7,138
6634,886,"{7: {0, 1, 2, 3, 4}, 3: {0, 1, 2, 3, 4, 5, 6, ...",1329,55.566516,0.719902,407,5,131
6636,433,"{5: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",1338,45.134927,0.508423,653,7,161
6667,174,"{4: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,...",1525,58.796935,0.426891,595,7,137


### 실제 유사한지 비교

In [72]:
TESTID['similar'] = TESTID['userID'].apply(lambda x : tdict[x]) 

In [73]:
compare = pd.merge(TESTID,validset,left_on='similar',right_on='userID',how='inner')

In [74]:
compare.columns =['userID_t', 'testId_t', 'count_t', 'elapsed_t', 'grade_acc_t',
       'last_count_t', 'last_grade_t', 'last_mid_t', 'similar', 'userID_v',
       'testId_v', 'count_v', 'elapsed_v', 'grade_acc_v', 'last_count_v',
       'last_grade_v', 'last_mid_v']

In [77]:
compare[['last_grade_t','last_grade_v','grade_acc_t','grade_acc_v','last_count_t','last_count_v','count_t','count_v','last_mid_t','last_mid_v']].head(20)

,last_grade_t,last_grade_v,grade_acc_t,grade_acc_v,last_count_t,last_count_v,count_t,count_v,last_mid_t,last_mid_v
0,2,2,0.500000,0.600000,4,5,15,13,159,124
1,3,3,0.500000,0.400000,4,5,15,14,111,126
2,4,4,0.214286,0.266667,14,15,15,20,133,180
3,4,4,0.250000,0.200000,4,5,15,21,170,168
4,5,5,1.000000,1.000000,4,4,15,13,178,193
5,5,5,0.666667,0.600000,3,5,15,60,193,159
6,5,5,0.666667,0.666667,3,6,15,67,193,162
7,2,2,0.400000,0.400000,10,10,16,42,170,161
8,4,4,0.733333,0.733333,15,15,16,19,130,180
9,6,6,0.000000,0.000000,3,5,16,25,67,171


In [78]:
compare[['last_grade_t','last_grade_v','grade_acc_t','grade_acc_v','last_count_t','last_count_v','count_t','count_v','last_mid_t','last_mid_v']].tail(20)

,last_grade_t,last_grade_v,grade_acc_t,grade_acc_v,last_count_t,last_count_v,count_t,count_v,last_mid_t,last_mid_v
724,7,7,0.376975,0.382353,443,442,1091,984,161,153
725,8,8,0.329577,0.332386,355,352,1103,529,117,113
726,1,1,0.877193,0.878378,285,296,1145,472,146,98
727,7,7,0.219761,0.223853,587,545,1145,872,139,163
728,2,2,0.693273,0.700885,877,565,1161,565,172,169
729,3,3,0.757508,0.752857,899,700,1218,700,187,155
730,8,8,0.633826,0.673282,609,655,1225,1028,132,138
731,8,8,0.232000,0.234783,125,115,1226,228,119,135
732,7,7,0.159011,0.175207,566,605,1228,1040,139,163
733,9,9,0.777465,0.756303,355,357,1238,656,23,74


In [79]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
DATA_PATH = '/opt/ml/input/data/train_dataset/train_data.csv'
df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [80]:
train_df = df[df["userID"].apply(lambda x : x not in val_index)]

In [81]:
valid_df = df[df["userID"].apply(lambda x : x in val_index)]

In [89]:
train_df.to_csv("/opt/ml/input/data/train_dataset/cv_train_data.csv",index=False)
valid_df.to_csv("/opt/ml/input/data/train_dataset/cv_valid_data.csv",index=False)

In [83]:
train = pd.read_csv("/opt/ml/input/data/train_dataset/cv_train_data.csv")

### grade 별 빈도

In [ ]:
# user data length
f = lambda x : len(set(x))
ff = lambda x : list(set(x))
grade_df = test_df.groupby('grade').agg({'userID':'count'})
grade_df.reset_index(inplace=True)
grade_df

grade_blk = blank.groupby('grade').agg({'userID':'count'})
grade_blk.reset_index(inplace=True)
grade_blk

pd.concat([grade_df,grade_blk], axis=1)

### Knowledge Tag 별 빈도

In [ ]:
f = lambda x : list(x)[-1]

kt_df = test_df.groupby('KnowledgeTag').agg({'grade' : f, 'userID':'count'})
kt_df = kt_df.sort_values(by=['grade', 'KnowledgeTag']).reset_index()
kt_df

In [ ]:
f = lambda x : list(x)[-1]

kt_blk = blank.groupby('KnowledgeTag').agg({'grade' : f, 'userID':'count'})
kt_blk.reset_index(inplace=True)
kt_blk

### Sequence Length 별 빈도

In [ ]:
# user data length
f = lambda x : len(set(x))
ff = lambda x : list(set(x))
ttest_df = test_df[test_df['answerCode']!=-1]

In [ ]:
ttest_df

In [ ]:
from collections import Counter

c = Counter([1,1,2])

In [ ]:
c.most_common()[0][0]

In [ ]:
from collections import Counter
def most(x):
    c = Counter(x)
    return c.most_common()[0][0]
    
user_df = ttest_df.groupby('userID').agg({'assessmentItemID':'count','grade':most})
user_df.reset_index(inplace=True)
user_df

In [ ]:
user_df['assessmentItemID'].plot(kind='hist')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df_cv = df[["userID","KnowledgeTag","answerCode","grade","elapsed","assessmentItemID"]]

In [ ]:
f = lambda x : list(x)[-1]
df_cv_user = df_cv.groupby('userID').agg({
    'elapsed': 'mean',
    'grade': f,
    'answerCode' : 'mean',
    'assessmentItemID':'count',
    'KnowledgeTag' : f
})
df_cv_user = df_cv_user.sort_values(by=['grade', 'KnowledgeTag']).reset_index(drop=True)

In [ ]:
df_cv_user

In [ ]:
grade_tdf = df_cv_user.groupby('grade').agg('count').reset_index()

In [ ]:
grade_tdf = grade_tdf[['grade','answerCode']]

In [ ]:
grade_tdf

In [ ]:
kt_df, kt_blk, grade_df, grade_blk

In [ ]:
sum(grade_blk['userID'])

In [ ]:
T = pd.merge(grade_blk,grade_tdf,on='grade')
T['pb'] = T['userID'] / sum(T['userID'])
T['max_user'] = sum(T['answerCode']) * T['pb']*0.2
T['max_user'] = T['max_user'].astype(int)
# 각 grade에서 1max user 개수 만큼 뽑을 것

In [ ]:
# 여부에 따라 join하고, 

In [ ]:
# sort하고 비율에 맞게 구간 자르고 
# ex) 5개씩, train, val index로 나눠서 저장
# categorical?은 어떻게 맞출까? 

In [ ]:
# class 별로 카테고리 N개 있으면, 